## Description
The objective of this competition is to create a machine learning model to detect fraudulent transactions.

Fraud detection is an important application of machine learning in the financial services sector. This solution will help Xente provide improved and safer service to its customers.

This competition is sponsored by Xente, Innovation Village, and insight2impact.

## Data
Xente is an e-commerce and financial service app serving 10,000+ customers in Uganda.

This dataset includes a sample of approximately 140,000 transactions that occurred between 15 November 2018 and 15 March 2019.

One of the challenges of fraud detection problems is that the data is highly imbalanced. 

Xente_variable_definitions.csv: Definition of the features per transaction
Training.csv: Transactions from 15 November 2018 to 13 February 2019, including whether or not each transaction is fraudulent. You will use this file to train your model.
Test.csv: Transactions from 13 February 2019 to 14 March 2019, not including whether or not each transaction is fraudulent. You will test your model on this file.
sample_submission.csv: is an example of what your submission file should look like. The order of the rows does not matter, but the names of the TransactionId must be correct. The value in FraudResult will be 1 for is a Fraud and 0 for is not a fraud.

## Evaluation
The error metric for this competition is the F1 score, which ranges from 0 (total failure) to 1 (perfect score). Hence, the closer your score is to 1, the better your model.

F1 Score: A performance score that combines both precision and recall. It is a harmonic mean of these two variables. Formula is given as: 2*Precision*Recall/(Precision + Recall)

Precision: This is an indicator of the number of items correctly identified as positive out of total items identified as positive. Formula is given as: TP/(TP+FP)

Recall / Sensitivity / True Positive Rate (TPR): This is an indicator of the number of items correctly identified as positive out of total actual positives. Formula is given as: TP/(TP+FN)

Where:

TP=True Positive
FP=False Positive
TN=True Negative
FN=False Negative

Info from Leaderboard: score to beat: 0,89

In [ ]:
import pandas as pd

In [ ]:
# Load the variable names
variable_meanings = pd.read_csv("data/variable meanings.csv")
pd.set_option('max_colwidth', 800)
variable_meanings

In [ ]:
# Load the data
data_test = pd.read_csv("data/test.csv")
data_train = pd.read_csv("data/training.csv")

In [ ]:
data_train.head()
#data_train.shape

In [ ]:
frauds = data_train.query('FraudResult == 1')

In [ ]:
data_train.query('CustomerId == "CustomerId_4406" ')
and AccountId == "AccountId_3957"

In [ ]:
data_train.query('CustomerId == "CustomerId_3957" ')
#and AccountId == "AccountId_4406"

In [ ]:
data_train.query('Amount<0 and CustomerId== "CustomerId_4841"')

In [ ]:
data_train.info()

## Results from Pandas Profile
1. There is only one CurrencyCode, that means there is no additional information provided by this feature ==> Drop CurrencyCode
2. There is only one CountryCode, that means there is no additional information provided by this feature ==> Drop CountryCode
3. TransactionIDs are all distinct, that means there is no additional information provided ==> Drop TransactionIds
4. TransactionStartTime consist of timestamps. For further analysis ==> group them into timeframes (use "datetime")
5. Amount contains + and - values (due to debit/credit) ==> we need to create a column with debit credit and transform "amount" to absolut values
6. Extreeeemely imbalanced target value ==> Oversampling? ==> read further information / links auf zindi nutzen
7. Definition for column  CustomerID and AccountId seems to be mixed up

In [ ]:
data_train.Amount.describe()

In [ ]:
data_train.Amount.value_counts()

In [ ]:
data_train.Amount.plot.hist(bins=50, ylim=(0,200))